In [ ]:
"""
keras-tuner : grid search / Random search / Bayes optimization / Hyper band
*Hyper band의 경우 NN에 적절하지 않을 수도 있음. 동시에 여러 모델을 학습하다가 
가장 효율이 떨어지는 모델을 절반씩 날리면서 가장 최후의 모델을 끝까지 학습 함으로서
minimum loss를 찾는 알고리즘인데, 실제 데이터/optimization algorithm에 따라서
학습 정도에 차이가 존재할 수 있다. 그래서 최근에는 베이즈 최적화와 하이퍼 밴드를
결합한 최적화 알고리즘을 많이 사용하기도 한다.
"""
import tensorflow as tf
from tensorflow import keras

# !pip install -q -U keras-tuner
import kerastuner as kt

# load dataset
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
    model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
    model.add(keras.layers.Dense(10))

    # Tune the learning rate for the optimizer 
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])

    return model

tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

# tuner = kt.RandomSearch(
#     build_model,
#     objective='val_loss',
#     max_trials=5)

# tuner = kt.BayesianOptimization(
#     build_model,
#     objective='val_loss',
#     max_trials=5)

# tuner = kt.sklearn()

class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)
        

In [ ]:
# 여기서 train/test 고정이 아니라 bagging / boosting 옵션 선택 했으면 좋겠다.
# https://github.com/keras-team/keras-tuner
# search 함수의 경우 fit과 똑같음

batch_size - 
dataset = tf.data.Dataset.from_tensor_slices(all_digits)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)
gan.fit(dataset.take(100), epochs=1)

tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""The hyperparameter search is complete. The optimal number of units in the first densely-connected
    layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
    is {best_hps.get('learning_rate')}.""")

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))